[![imagenes](imagenes/pythonista.png)](https://pythonista.io)

# Desarrollo de una servidor de API REST con *Flask-SQLAlchemy*.

En este capítulo se reutilizará el código de la notebook [12_servidor_api_rest.ipynb](12_servidor_api_rest.ipynb), para realizar operaciones en la base de datos creada en la notebook [14_orm_con_flask_sqlalchemy.ipynb](14_orm_con_flask_sqlalchemy.ipynb).

**Advertencia:** Es necesario que se haya creado correctamente la base de datos localizada en [data/alumnos.db]([data/alumnos.db).

## Importación de módulos y datos.

In [ ]:
from flask import Flask, jsonify, request, abort
from data import campos, orden
from json import loads
from flask_sqlalchemy import SQLAlchemy

## Creación de la aplicación e inicio de la sesión con la base de datos.

In [ ]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///data/alumnos.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

## Definición de la clase *Alumno*.

In [ ]:
class Alumno(db.Model):
    __tablename__ = 'alumnos'
    cuenta = db.Column(db.Integer, primary_key=True)
    nombre = db.Column(db.String(50))
    primer_apellido = db.Column(db.String(50))
    segundo_apellido = db.Column(db.String(50))
    carrera = db.Column(db.String(50))
    semestre = db.Column(db.Integer)
    promedio = db.Column(db.Float)
    al_corriente = db.Column(db.Boolean)

## Función de conversión de los objetos a objetos tipo _dict_.

Para poder enviar los datos a través de la API es necesario convertir los atributos de los objetos instanciados de la clase *Alumno* en campos de una objeto *dict*.

In [ ]:
def convierte(cuenta, alumno, orden):
    respuesta = {'cuenta': cuenta}
    for campo in orden:
        respuesta[campo] = getattr(alumno, campo.lower().replace(' ', '_'))
    return respuesta

## Funciones de validación de datos.

Estas funciones se definen de forma idéntica al capítulo [12_servidor_api_rest.ipynb](12_servidor_api_rest.ipynb).

In [ ]:
def es_tipo(dato, tipo):
    if tipo == str:
        return True
    else:
        try: 
            return tipo(dato) is dato
        except:
            return False
        
def reglas(dato, campo):
    from data import carreras
    if campo == "Carrera" and dato not in carreras:
        return False
    elif campo == "Semestre" and dato < 1:
        return False
    elif campo == "Promedio" and (dato < 0 or dato > 10):
        return False
    elif (campo in ("Nombre", "Primer Apellido") and (dato == "")):
        return False
    else:
        return True           
    
def valida(dato, campo):
    return es_tipo(dato, campos[campo][0]) and reglas(dato, campo)

## La función *recurso_completo()*.
Esta función fue modificada para poder gestionar objetos instanciados de *Alumno* y añadir registros a la base de datos.

In [ ]:
def recurso_completo(cuenta, peticion):
    try:
        candidato = Alumno(cuenta=int(cuenta))
        peticion = loads(peticion)
        if (set(peticion)).issubset(set(orden)):                    
            for campo in orden:
                if not campos[campo][1] and campo not in peticion:
                    setattr(candidato, campo.lower().replace(' ', '_'), '')
                elif valida(peticion[campo], campo):
                    setattr(candidato, campo.lower().replace(' ', '_'), peticion[campo])    
                else:
                    abort(400)
        else:
            abort(400)
    except:
        abort(400)
    db.session.add(candidato)
    db.session.commit()                        
    return jsonify(convierte(cuenta, candidato, orden))

## Las funciones de vista.

Las funciones de vista conservan una estructura lógica idéntica, pero fueron modificadas para manipular los objetos instanciados de *Alumno* y realizar operaciones de búsqueda, modificación y eliminación de registros en la base de datos.

In [ ]:
@app.route('/api/')
def ruta_api():
    lista = Alumno.query.filter(Alumno.cuenta).all()
    return jsonify([convierte(registro.cuenta, registro, orden) for registro in lista])
    

@app.route('/api/<cuenta>', methods=['GET', 'POST', 'PUT', 'PATCH', 'DELETE'])
def api(cuenta):
    
    if request.method == 'GET':
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno == None:
            abort(404)
        else:
            return jsonify(convierte(cuenta, alumno, orden))
            
            
    if request.method == 'DELETE':
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno == None:
            abort(404)
        else:
            db.session.delete(alumno)
            db.session.commit()
            return jsonify(convierte(cuenta, alumno, orden))
        
        
    if request.method == 'POST':
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno == None:
            return recurso_completo(cuenta, request.data)
        else:
            abort(409)
            
            
    if request.method == 'PUT':
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno == None:
            abort(404)
        else:
            db.session.delete(alumno)
            db.session.commit()
            return recurso_completo(cuenta, request.data)

    if request.method == 'PATCH':               
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno == None:
            abort(404)
        else:
            try:
                peticion = loads(request.data)
                if (set(peticion)).issubset(set(orden)):
                    for campo in peticion:
                        dato = peticion[campo]
                        if valida(dato, campo):
                            setattr(alumno, campo.lower().replace(' ', '_'), dato)
                        else:
                            abort(400)
                else:
                    abort(400)
            except:
                abort(400)
            db.session.add(alumno)
            db.session.commit()                        
            return jsonify(convierte(cuenta, alumno, orden))

In [ ]:
app.run('0.0.0.0')

### Notas:

* **No reinicie o detenga el kernel de la notebook hasta que haya terminado todas las sesiones los clientes.**
* Debido a que el código de la celda de arriba levanta el servidor de Flask, ésta se ejecutará indefinidamente y desplegará los mensajes de respuesta a las peticiones de los clientes que se conecten.
* La notebook [13_cliente_api_rest.ipynb](13_cliente_api_rest.ipynb) contiene al cliente para este servidor.

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2018.</p>